In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

# <font size="6">1. Khai báo tên file</font>

In [2]:
issues_file_name = r'D:\projects\Python\BTL\raw_data\all_issues_data.json'

# <font size="6">2. Mở dữ liệu tất cả các issue</font>

In [3]:
with open(issues_file_name, 'r', encoding='utf-8') as file:
    issues_data = json.load(file)
df = pd.json_normalize(issues_data)
df = df[['id', 'url', 'user.login', 'title', 'state', 'comments', 'created_at', 'closed_at', 'pull_request.url']]

In [4]:
# Lưu dataframe thành file excel
df.to_excel(r'D:\projects\Python\BTL\cleaned_data\cleaned_all_issues_and_pulls_data.xlsx')

# <font size="6">3. Tạo thêm hai dataframe để phân biệt issue và pull request</font>

In [5]:
num_of_issues_and_pull = len(df)

all_issues_df = df[df['pull_request.url'].isna()]
all_issues_df = all_issues_df[['id', 'url', 'user.login', 'title', 'state', 'comments', 'created_at', 'closed_at']]

all_pull_df = df[df['pull_request.url'].isna() == False]
all_pull_df = all_pull_df[['id', 'url', 'user.login', 'title', 'state', 'comments', 'created_at', 'closed_at', 'pull_request.url']]

In [6]:
# Lưu dataframe thành file excel
all_issues_df.to_excel(r'D:\projects\Python\BTL\cleaned_data\cleaned_all_issues_data.xlsx')
all_pull_df.to_excel(r'D:\projects\Python\BTL\cleaned_data\cleaned_all_pulls_data.xlsx')

# <font size="6">4. Phân tích issue<font>

<font size="4">4.1. Số lượng issue</font>

In [7]:
num_of_issues = len(all_issues_df)
num_of_issues

26473

<font size="4">4.2. Số lượng issue mở</font>

In [8]:
open_issues = pd.DataFrame(all_issues_df[all_issues_df['state'] == 'open'])
open_issues.reset_index(drop = True, inplace = True)
len(open_issues)

3627

<font size="4">4.3. Số lượng issue đóng</font>

In [9]:
closed_issues = pd.DataFrame(all_issues_df[all_issues_df['state'] == 'closed'])
closed_issues.reset_index(drop = True, inplace = True)
len(closed_issues)

22846

<font size="4">4.4. Thêm cột thời gian để đóng 1 issue</font>

In [10]:
time_for_issue = []
for i in range(len(closed_issues)):
    closed_issue = closed_issues.iloc[i]
    start_time = datetime.strptime(closed_issue['created_at'], '%Y-%m-%dT%H:%M:%SZ')
    end_time = datetime.strptime(closed_issue['closed_at'], '%Y-%m-%dT%H:%M:%SZ')
    time_for_issue.append(round((end_time - start_time).seconds / 60))
time_for_issue_df = pd.Series(time_for_issue, name = 'time_for_issue')
closed_issues = pd.concat([closed_issues, time_for_issue_df], axis = 1)

<font size="4">4.5. Thời gian trung bình để đóng 1 issue</font>

In [11]:
average_time_for_a_closed_issue = round(sum(time_for_issue) / (len(time_for_issue)), 2)
average_time_for_a_closed_issue

589.03

<font size="4">4.6. Các thông tin khác về thời gian để đóng 1 issue</font>

In [12]:
# Các thông tin khác
closed_issues['time_for_issue'].describe()

count    22846.000000
mean       589.032566
std        456.968204
min          0.000000
25%        164.000000
50%        499.000000
75%       1002.000000
max       1440.000000
Name: time_for_issue, dtype: float64

<font size="4">4.7. Số issue có thời gian hoàn thành bằng 0</font>

In [13]:
len(closed_issues[closed_issues['time_for_issue'] == 0])

37

<font size="4">4.8. Issue có nhiều lượt bàn luận nhất</font>

In [14]:
pd.set_option('display.max_colwidth', 200)
sort_by_num_of_comments_issue_df = df.sort_values(by = 'comments', ascending = False)
sort_by_num_of_comments_issue_df[['url', 'comments']].head()

,url,comments
54723,https://api.github.com/repos/pandas-dev/pandas/issues/4164,477
55494,https://api.github.com/repos/pandas-dev/pandas/issues/3393,298
54724,https://api.github.com/repos/pandas-dev/pandas/issues/4163,176
54503,https://api.github.com/repos/pandas-dev/pandas/issues/4384,175
51670,https://api.github.com/repos/pandas-dev/pandas/issues/7217,156


<font size="4">4.9. Từ xuất hiện nhiều nhất trong các title</font>

In [15]:
words = [x.split()[0] for x in df['title']]
words_with_no_duplication = pd.DataFrame(list({x:words.count(x)for x in words}.items()), columns = ['Words', 'Time'] )
words_with_no_duplication.sort_values(by = 'Time', ascending = False, inplace = True, ignore_index = True)
words_with_no_duplication.head()

,Words,Time
0,BUG:,10919
1,DOC:,5074
2,ENH:,3331
3,CLN:,2396
4,TST:,2227


<font size="4">4.10.1. Hàm trả về số issue được tạo ra trong 1 tháng</font>

In [16]:
def get_created_issue_in_a_specific_month(month):
    month_search = datetime.strftime(datetime.strptime(month, '%m/%Y'), '%Y-%m')
    month_mask = all_issues_df['created_at'].str.contains(month_search)
    issues_in_a_specific_month = pd.DataFrame(all_issues_df[month_mask])
    issues_in_a_specific_month.reset_index(drop = True, inplace = True)
    return issues_in_a_specific_month['url']

<font size="4">4.10.2. Issue được tạo ra mỗi tháng</font>

In [17]:
months = []
for year in range(2010, 2025):
    for month in range(1, 13):
        if year == 2024 and month > 6:
            break
        months.append(f"{month:02d}/{year}")
        
month_series1 = pd.Series(index = months)
for month in months:
    month_series1[month] = len(get_created_issue_in_a_specific_month(month))    

name = month_series1.index
value = month_series1.values
x = np.arange(len(months))

fig, ax = plt.subplots(figsize=(200, 50))
ax.bar( name, value)
ax.set_xticks(name)
ax.set_xticklabels(name, rotation = 90, ha = 'right')
for i in range(len(months)):
    plt.text(i, int(value[i]) + 0.5, int(value[i]), ha = 'center')
plt.ylabel('Commit')
plt.xlabel('Issues have created')
plt.show()

<font size="4">4.11.1. Hàm trả về số issue đóng lại trong 1 tháng</font>

In [18]:
def get_closed_issue_in_a_specific_month(month):
    month_search = datetime.strftime(datetime.strptime(month, '%m/%Y'), '%Y-%m')
    month_mask = closed_issues['closed_at'].str.contains(month_search)
    issues_in_a_specific_month = pd.DataFrame(closed_issues[month_mask])
    issues_in_a_specific_month.reset_index(drop = True, inplace = True)
    return issues_in_a_specific_month['url']

<font size="4">4.11.2. Số issue đóng lại trong 1 tháng</font>

In [19]:
# Issue đóng lại mỗi tháng
months = []
for year in range(2010, 2025):
    for month in range(1, 13):
        if year == 2024 and month > 6:
            break
        months.append(f"{month:02d}/{year}")
        
month_series2 = pd.Series(index = months)
for month in months:
    month_series2[month] = len(get_closed_issue_in_a_specific_month(month))    

name = month_series2.index
value = month_series2.values
x = np.arange(len(months))

fig, ax = plt.subplots(figsize=(200, 50))
ax.bar( name, value)
ax.set_xticks(name)
ax.set_xticklabels(name, rotation = 90, ha = 'right')
for i in range(len(months)):
    plt.text(i, int(value[i]) + 0.5, int(value[i]), ha = 'center')
plt.ylabel('Commit')
plt.xlabel('Issues have created')
plt.show()

<font size="4">4.12. Sự khác nhau giữa biểu đồ issue đóng và issue mở</font>

In [20]:

fig, ax = plt.subplots(figsize=(200, 50))
name = month_series2.index
value1 = month_series1.values
value2 = month_series2.values

#Biểu đồ issue mở
ax.bar( name, value1, label = 'Open issue')

# Biểu đồ issue đóng
ax.plot(name, value2, color='red', label = 'Closed issue', linewidth=10)

ax.set_xlabel('Comparison of open issues and close issues')
ax.set_ylabel('Issue')


ax.legend(loc='upper left', fontsize=200)

plt.show()